<a href="https://colab.research.google.com/github/Carinaaa/ML-Learning-Path/blob/intro-LLM/LLMs_summary_openAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.openai.com/api-keys
https://stackoverflow.com/questions/66631333/how-do-i-set-environment-variables-in-google-colab

https://github.com/ed-donner/llm_engineering


In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [4]:
from google.colab import userdata
import os
# load_dotenv() # load the .env file if you have it, i don't and I use secrets from GC
openai_key = userdata.get('OPENAI_API_KEY') # get the secret by the name
os.environ['OPENAI_API_KEY'] = openai_key # set it as an env var
openai = OpenAI() # create model using the env var

In [5]:
message = "Hello, GPT! This is my first ever message to you! Hi!"

response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])

print(response.choices[0].message.content)

Hello! Welcome! I'm glad to hear from you. How can I assist you today?


In [6]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
History_of_Christianity_wiki = Website("https://en.wikipedia.org/wiki/History_of_Christianity")
print(History_of_Christianity_wiki.title)
print(History_of_Christianity_wiki.text)

History of Christianity - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Special pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
Early Christianity (c. 27 – fourth century)
Toggle Early Christianity (c. 27 – fourth century) subsection
1.1
First century
1.2
Ante-Nicene period (100–312)
2
Late antiquity (313 – c. 600)
Toggle Late antiquity (313 – c. 600) subsection
2.1
Geographical spread
2.2
Religious violence
2.3
Heresies, schisms and councils
2.4
After 476
3
Early Middle Ages (c. 600–1000)
Toggle Early Middle Ages (c. 600–1000) subsection
3.1
Monasticism and art
3.2
Regional differences
4
High Middle Ages (c. 1000–1300)
Toggle High Middle Ages (c. 1000–1300

## Types of prompts

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [9]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [10]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
print(user_prompt_for(History_of_Christianity_wiki))

You are looking at a website titled History of Christianity - Wikipedia
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Special pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
Early Christianity (c. 27 – fourth century)
Toggle Early Christianity (c. 27 – fourth century) subsection
1.1
First century
1.2
Ante-Nicene period (100–312)
2
Late antiquity (313 – c. 600)
Toggle Late antiquity (313 – c. 600) subsection
2.1
Geographical spread
2.2
Religious violence
2.3
Heresies, schisms and councils
2.4
After 476

In [13]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, I don't know. Maybe it's 22? Just kidding! It's 4. You know, basic math and all that.


In [14]:
# And now let's build useful messages for GPT-4o-mini, using a function

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [17]:
print(messages_for(History_of_Christianity_wiki))

[{'role': 'system', 'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'}, {'role': 'user', 'content': 'You are looking at a website titled History of Christianity - Wikipedia\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nJump to content\nMain menu\nMain menu\nmove to sidebar\nhide\nNavigation\nMain page\nContents\nCurrent events\nRandom article\nAbout Wikipedia\nContact us\nContribute\nHelp\nLearn to edit\nCommunity portal\nRecent changes\nUpload file\nSpecial pages\nSearch\nSearch\nAppearance\nDonate\nCreate account\nLog in\nPersonal tools\nDonate\nCreate account\nLog in\nPages for logged out editors\nlearn more\nContributions\nTalk\nContents\nmove to sidebar\nhide\n(Top)\n1\nEarly Christianity (c. 27 – fourth century)\nToggle Early Christian

In [18]:
# Time to bring it together: call the OpenAI API.

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
print(summarize("https://en.wikipedia.org/wiki/History_of_Christianity"))

# Summary of "History of Christianity - Wikipedia"

The Wikipedia article on the **History of Christianity** provides a comprehensive overview of Christianity from its origins to the modern day. Here’s a brief summary of the key sections outlined in the article:

## 1. **Early Christianity (c. 27 – fourth century)**
- Development begins with Jesus Christ, whose teachings and resurrection fostered a small movement which morphed into a larger, organized religion by the fourth century.
- The first Christians were primarily Jewish, with rapid urban growth in the Roman Empire.
- Key events include the Edict of Milan in 313 which legalized Christianity.

## 2. **Late Antiquity (313 – c. 600)**
- Christianity gained state support, leading to a flourishing period for Christian art, architecture, and theology.
- The establishment of important councils, such as Nicaea, addressed heresies and schisms.

## 3. **Early Middle Ages (c. 600–1000)**
- Marked by the preservation of Christian texts by mo

In [21]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [22]:
display_summary("https://en.wikipedia.org/wiki/History_of_Christianity")

# Summary of the "History of Christianity" - Wikipedia

The Wikipedia article on the **History of Christianity** provides a comprehensive overview of the development and spread of Christianity from its inception to the present day. It is organized chronologically and thematically, covering significant periods, events, and figures in Christian history.

## Key Sections:

### 1. **Early Christianity (c. 27 – 4th Century)**
- Begins with Jesus Christ and his followers, emphasizing their proclamations and early rituals such as baptism and the Eucharist.
- Discusses the growth of Christian communities in urban settings and highlights significant figures like the Apostle Paul.
- Explores the Ante-Nicene period (100-312 CE), detailing the spread of Christianity across the Roman Empire, its theological developments, heresies, and the roles of early church leaders.

### 2. **Late Antiquity (313 – c. 600)**
- Covers the legalization of Christianity through Constantine's Edict of Milan and the subsequent establishment of Christianity as the state religion in the Roman Empire.
- Describes the geographical spread of Christianity and various theological disputes and councils.

### 3. **Middle Ages (c. 600–1300)**
- Discusses the establishment of monasticism, the influence of prominent church figures like Augustine, and the challenges faced, including the Crusades.
- Examines the interactions and conflicts between different Christian factions, especially during the East-West Schism in 1054.

### 4. **Renaissance and Reformation (c. 1300–1650)**
- Discusses the impact of the Renaissance on Christian thought and art and the rise of Reformation movements led by figures like Martin Luther.
- Explores the social, political, and theological implications of the Reformation and various responses from the Catholic Church, known as the Counter-Reformation.

### 5. **Modern Period (1650–1945)**
- Focuses on changes in religious thought during periods of Enlightenment, the rise of secularization, and the expansion of Christianity during the colonial era.
- Addresses the development of various denominations and movements, including Pentecostalism and fundamentalism.

### 6. **After World War II**
- Discusses the global shift in Christianity's demographic center towards the Global South, particularly in Africa and Latin America.
- Covers the emergence of new movements, the challenges of secularism, and ongoing issues of discrimination and persecution of Christians around the world.

## Conclusion
The article emphasizes that Christianity remains the world's largest religion today, with a diverse global presence that includes numerous denominations and traditions, influencing various aspects of culture, politics, and society across different regions.

This overview encapsulates the essential milestones and diverse pathways of Christianity throughout history, underscoring its profound impact on global affairs and human experience.